# 🏢 Priorisation des Bâtiments à Risque - Montréal

**Projet VILLE_IA** - Institut de la résilience et de l'innovation urbaine (IRIU)

## Approche Innovante: SANS Géomatique

Ce notebook démontre comment identifier les bâtiments prioritaires pour rénovation énergétique et adaptation climatique **sans utiliser d'outils géomatiques**.

### Notre Innovation
- ✅ Utilisation de l'intelligence des codes postaux
- ✅ Analyse textuelle des adresses
- ✅ Machine Learning multi-critères
- ✅ Proxys de vulnérabilité par arrondissement

---

## 1. Configuration et Import

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Imports réussis")

## 2. Chargement des Données

Nous chargeons les données disponibles sur les bâtiments municipaux de Montréal.

In [ ]:
# Charger les données de bâtiments
buildings = pd.read_csv('data/batiments-municipaux.csv')

print(f"📊 Nombre de bâtiments: {len(buildings):,}")
print(f"\n📋 Colonnes disponibles:")
print(buildings.columns.tolist())

# Aperçu
buildings.head()

## 3. Exploration Rapide

In [ ]:
# Statistiques descriptives
print("📈 Statistiques Descriptives:")
print(buildings.describe())

# Valeurs manquantes
print("\n❓ Valeurs Manquantes:")
missing = buildings.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

In [ ]:
# Distribution par arrondissement
fig = px.bar(
    buildings['boroughName'].value_counts().reset_index(),
    x='boroughName',
    y='count',
    title='Nombre de Bâtiments par Arrondissement',
    labels={'boroughName': 'Arrondissement', 'count': 'Nombre de Bâtiments'}
)
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

## 4. Exécution du Pipeline Complet

### Étape 1: Matching Intelligent (Sans Géomatique)

In [ ]:
# Importer notre classe de matching intelligent
import sys
sys.path.append('.')

from importlib import import_module
matching = import_module('02_intelligent_matching')
IntelligentMatcher = matching.IntelligentMatcher

# Créer le matcher
matcher = IntelligentMatcher()

# Enrichir avec intelligence des codes postaux
buildings_enriched = matcher.enrich_with_postal_code_intelligence(buildings.copy())

print("✅ Enrichissement complété")
print(f"\n📍 Colonnes ajoutées:")
print("  - postal_prefix: Préfixe du code postal (ex: H3)")
print("  - postal_flood_risk: Risque inondation basé sur le code postal")
print("  - postal_heat_risk: Risque chaleur basé sur le code postal")

# Exemple
buildings_enriched[['buildingName', 'address', 'postal_prefix', 'postal_flood_risk', 'postal_heat_risk']].head(10)

### Visualisation des Risques par Code Postal

In [ ]:
# Agrégation par code postal
postal_risks = buildings_enriched.groupby('postal_prefix').agg({
    'postal_flood_risk': 'mean',
    'postal_heat_risk': 'mean',
    'buildingid': 'count'
}).reset_index()

postal_risks.columns = ['Code Postal', 'Risque Inondation', 'Risque Chaleur', 'Nombre de Bâtiments']

# Graphique
fig = go.Figure()

fig.add_trace(go.Bar(
    x=postal_risks['Code Postal'],
    y=postal_risks['Risque Inondation'],
    name='Risque Inondation',
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    x=postal_risks['Code Postal'],
    y=postal_risks['Risque Chaleur'],
    name='Risque Chaleur',
    marker_color='orange'
))

fig.update_layout(
    title='Risques Climatiques par Zone (Code Postal)',
    xaxis_title='Préfixe Code Postal',
    yaxis_title='Score de Risque',
    barmode='group',
    height=500
)

fig.show()

## 5. Modèle de Priorisation ML

### Calcul des Scores de Priorité

In [ ]:
# Importer le modèle de priorisation
ml_model = import_module('03_ml_prioritization_model')
BuildingRiskPrioritizer = ml_model.BuildingRiskPrioritizer

# Initialiser le modèle
model = BuildingRiskPrioritizer()

# Créer les features
features = model.create_feature_matrix(buildings_enriched)

print("\n✅ Features créées:")
print(features.columns.tolist())
print("\n📊 Statistiques des features:")
features.describe()

In [ ]:
# Calculer les scores de priorité
buildings_enriched['priority_score'] = model.calculate_priority_score(features)

# Classifier par niveau
buildings_enriched['priority_level'] = pd.cut(
    buildings_enriched['priority_score'],
    bins=[0, 40, 60, 80, 100],
    labels=['Low', 'Medium', 'High', 'Critical']
)

# Ajouter les features individuelles
for col in features.columns:
    buildings_enriched[f'score_{col}'] = features[col]

print("✅ Scores calculés")
print("\n📊 Distribution des niveaux de priorité:")
print(buildings_enriched['priority_level'].value_counts())

## 6. Analyse des Résultats

### Top 20 Bâtiments Prioritaires

In [ ]:
# Top 20
top_20 = buildings_enriched.nlargest(20, 'priority_score')[[
    'buildingName', 'address', 'boroughName', 
    'priority_score', 'priority_level',
    'score_energy_risk', 'score_climate_risk', 'score_social_vulnerability'
]]

print("🏆 TOP 20 BÂTIMENTS PRIORITAIRES:")
top_20

### Visualisation des Scores Multi-Dimensionnels

In [ ]:
# Scatter plot 3D
fig = px.scatter_3d(
    buildings_enriched,
    x='score_energy_risk',
    y='score_climate_risk',
    z='score_social_vulnerability',
    color='priority_level',
    hover_data=['buildingName', 'boroughName'],
    title='Analyse Multi-Dimensionnelle des Risques',
    labels={
        'score_energy_risk': 'Risque Énergétique',
        'score_climate_risk': 'Risque Climatique',
        'score_social_vulnerability': 'Vulnérabilité Sociale'
    },
    color_discrete_map={
        'Critical': '#ff4444',
        'High': '#ff8c00',
        'Medium': '#ffd700',
        'Low': '#90ee90'
    }
)

fig.update_layout(height=700)
fig.show()

### Analyse par Arrondissement

In [ ]:
# Statistiques par arrondissement
borough_analysis = buildings_enriched.groupby('boroughName').agg({
    'priority_score': ['mean', 'max', 'min'],
    'buildingid': 'count',
    'score_energy_risk': 'mean',
    'score_climate_risk': 'mean',
    'score_social_vulnerability': 'mean'
}).round(2)

borough_analysis.columns = [
    'Score Moyen', 'Score Max', 'Score Min', 'Nb Bâtiments',
    'Risque Énergie', 'Risque Climat', 'Vulnérabilité Sociale'
]

borough_analysis = borough_analysis.sort_values('Score Moyen', ascending=False)

print("📊 ANALYSE PAR ARRONDISSEMENT:")
borough_analysis

In [ ]:
# Heatmap
fig = px.imshow(
    borough_analysis[['Score Moyen', 'Risque Énergie', 'Risque Climat', 'Vulnérabilité Sociale']],
    labels=dict(x="Métriques", y="Arrondissement", color="Score"),
    title="Carte de Chaleur: Risques par Arrondissement",
    aspect="auto",
    color_continuous_scale="RdYlGn_r"
)

fig.update_layout(height=600)
fig.show()

## 7. Estimation d'Impact

### Potentiel de Réduction de GES

In [ ]:
# Calculer le potentiel GES
buildings_enriched['estimated_ges_reduction'] = (
    buildings_enriched['buildingArea'].fillna(
        buildings_enriched['builtArea'].fillna(1000)
    ) / 100 *
    buildings_enriched['score_energy_risk'] *
    buildings_enriched['score_age_risk'] *
    2.5  # Facteur de conversion
)

# Statistiques
total_potential = buildings_enriched['estimated_ges_reduction'].sum()
top_100_potential = buildings_enriched.nlargest(100, 'priority_score')['estimated_ges_reduction'].sum()
critical_potential = buildings_enriched[
    buildings_enriched['priority_level'] == 'Critical'
]['estimated_ges_reduction'].sum()

print("🌱 POTENTIEL DE RÉDUCTION GES:")
print(f"\n  Total (tous bâtiments): {total_potential:,.0f} tonnes CO₂/an")
print(f"  Top 100 bâtiments: {top_100_potential:,.0f} tonnes CO₂/an ({top_100_potential/total_potential*100:.1f}%)")
print(f"  Bâtiments critiques: {critical_potential:,.0f} tonnes CO₂/an ({critical_potential/total_potential*100:.1f}%)")

In [ ]:
# Courbe cumulative d'impact
buildings_sorted = buildings_enriched.sort_values('priority_score', ascending=False).reset_index(drop=True)
buildings_sorted['cumulative_ges'] = buildings_sorted['estimated_ges_reduction'].cumsum()
buildings_sorted['cumulative_pct'] = buildings_sorted['cumulative_ges'] / total_potential * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(buildings_sorted) + 1)),
    y=buildings_sorted['cumulative_pct'],
    mode='lines',
    name='Impact Cumulatif',
    line=dict(color='green', width=3)
))

# Ligne des 100 premiers
fig.add_vline(x=100, line_dash="dash", line_color="red",
              annotation_text="Top 100", annotation_position="top right")

fig.update_layout(
    title='Courbe d\'Impact Cumulatif: Réduction GES',
    xaxis_title='Nombre de Bâtiments Rénovés (par ordre de priorité)',
    yaxis_title='% du Potentiel Total de Réduction GES',
    height=500
)

fig.show()

print(f"\n💡 Insight: En rénovant les {100} bâtiments les plus prioritaires,")
print(f"   on capture {top_100_potential/total_potential*100:.1f}% du potentiel total de réduction.")

## 8. Analyse par Âge de Bâtiment

In [ ]:
# Créer des catégories d'âge
current_year = 2024
buildings_enriched['building_age'] = current_year - buildings_enriched['buildingConstrYear']
buildings_enriched['age_category'] = pd.cut(
    buildings_enriched['building_age'],
    bins=[0, 20, 40, 60, 100, 200],
    labels=['< 20 ans', '20-40 ans', '40-60 ans', '60-100 ans', '> 100 ans']
)

# Analyse par âge
age_analysis = buildings_enriched.groupby('age_category').agg({
    'priority_score': 'mean',
    'buildingid': 'count',
    'estimated_ges_reduction': 'sum'
}).round(2)

age_analysis.columns = ['Score Moyen', 'Nombre', 'Potentiel GES Total']

print("🏗️ ANALYSE PAR ÂGE DE BÂTIMENT:")
age_analysis

In [ ]:
# Visualisation
fig = go.Figure()

fig.add_trace(go.Bar(
    x=age_analysis.index,
    y=age_analysis['Score Moyen'],
    name='Score Moyen',
    marker_color='lightblue',
    yaxis='y'
))

fig.add_trace(go.Scatter(
    x=age_analysis.index,
    y=age_analysis['Potentiel GES Total'],
    name='Potentiel GES',
    marker_color='green',
    yaxis='y2',
    mode='lines+markers',
    line=dict(width=3)
))

fig.update_layout(
    title='Priorité et Impact par Âge de Bâtiment',
    xaxis_title='Catégorie d\'Âge',
    yaxis_title='Score de Priorité Moyen',
    yaxis2=dict(
        title='Potentiel GES Total (tonnes CO₂/an)',
        overlaying='y',
        side='right'
    ),
    height=500
)

fig.show()

## 9. Export des Résultats

In [ ]:
# Sauvegarder les résultats complets
buildings_enriched.to_csv('output_buildings_prioritized.csv', index=False, encoding='utf-8-sig')
print("✅ Résultats complets sauvegardés: output_buildings_prioritized.csv")

# Sauvegarder le Top 100
top_100 = buildings_enriched.nlargest(100, 'priority_score')
top_100.to_csv('output_top_100_priorities.csv', index=False, encoding='utf-8-sig')
print("✅ Top 100 sauvegardé: output_top_100_priorities.csv")

# Rapport sommaire
print("\n" + "="*80)
print("RÉSUMÉ FINAL")
print("="*80)
print(f"\n📊 Bâtiments analysés: {len(buildings_enriched):,}")
print(f"\n🎯 Distribution des priorités:")
for level in ['Critical', 'High', 'Medium', 'Low']:
    count = len(buildings_enriched[buildings_enriched['priority_level'] == level])
    pct = count / len(buildings_enriched) * 100
    print(f"   {level:10s}: {count:4d} ({pct:5.1f}%)")

print(f"\n🌱 Potentiel total de réduction GES: {total_potential:,.0f} tonnes CO₂/an")
print(f"   Top 100 bâtiments: {top_100_potential:,.0f} tonnes CO₂/an")

print("\n🏆 Top 5 Bâtiments:")
for i, row in buildings_enriched.nlargest(5, 'priority_score').iterrows():
    print(f"   {row['buildingName'][:50]:50s} - Score: {row['priority_score']:.1f}")

print("\n" + "="*80)
print("✅ ANALYSE COMPLÉTÉE")
print("="*80)
print("\nProchaines étapes:")
print("  1. Lancer le dashboard web: streamlit run 04_web_dashboard.py")
print("  2. Consulter la méthodologie: METHODOLOGY.md")
print("  3. Planifier les interventions basées sur les résultats")

## 10. Conclusion

### Ce que nous avons démontré:

✅ **Approche sans géomatique fonctionnelle**
- Remplacement des coordonnées par l'intelligence des codes postaux
- Proxys géographiques efficaces
- Résultats cohérents et actionnables

✅ **Priorisation multi-critères**
- 40% Impact énergétique / GES
- 30% Risques climatiques
- 20% Équité sociale
- 10% Potentiel d'impact

✅ **Solution accessible et reproductible**
- Pas besoin de logiciels SIG
- Code Python standard
- Pipeline automatisé
- Résultats transparents

### Prochaines Étapes pour les Municipalités:

1. **Court terme (3 mois)**: Auditer les bâtiments critiques
2. **Moyen terme (1 an)**: Rénover le Top 20
3. **Long terme (3-5 ans)**: Programme systématique Top 100

---

**📧 Questions?** Consultez METHODOLOGY.md ou contactez le projet VILLE_IA

**🌐 Dashboard Web:** `streamlit run 04_web_dashboard.py`
